

-# Install packages

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install statsmodels
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install scikit-learn
# !pip install imblearn
# !pip install xgboost
# !pip install lightgbm
# !pip install keras
# !pip install dill

# Import libraries

In [2]:
## Data loading and manipulation
from pandas import read_csv
from pandas import DataFrame
from pandas import to_datetime
from pandas import to_numeric
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
from pandas import CategoricalDtype
from numpy import nan as nan_value
from pandas import concat
from pandas import cut
from numpy import quantile
from numpy import linspace
from numpy import concatenate
from numpy import inf as np_inf
from fuzzywuzzy import process

## EDA
import matplotlib.pyplot as plt
from pandas import DatetimeIndex
import seaborn as sns
from IPython.display import Image

## Feature Engineering
from scipy import stats
from numpy import log
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data pre-processing
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

## Model building
from sklearn.model_selection import RandomizedSearchCV

# Linear models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Tree-based models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Support Vector Machines
from sklearn.svm import SVC

# Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

# Naive Bayes
from sklearn.naive_bayes import GaussianNB

# Neural Networks
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.optimizers import Adam

## Model Evaluation
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

## Saving model
import joblib
import pickle

## Other imports
from os import listdir
from os import makedirs
from os.path import exists
from os import environ
import time
from datetime import datetime
import time
from datetime import timedelta
import dill

# Project configurations

In [23]:
DATA_DIR = 'data'
EDA_REPORTS_DIR = 'eda'
TRANSFORMATIONS_DIR = 'transformations'
CROSS_VALIDATION_DIR = 'cv_results'
MODEL_FITTING_RESULTS_DIR = 'model_fitting'
MODEL_DEPLOYMENT_DIR = 'deployed_models'

In [24]:
makedirs(DATA_DIR, exist_ok=True)
makedirs(EDA_REPORTS_DIR, exist_ok=True)
makedirs(MODEL_DEPLOYMENT_DIR, exist_ok=True)
makedirs(TRANSFORMATIONS_DIR, exist_ok=True)
makedirs(CROSS_VALIDATION_DIR, exist_ok=True)
makedirs(MODEL_FITTING_RESULTS_DIR, exist_ok=True)

# Data transformation module

In [1]:
class TransformationModule:
    def __init__(self):
        self.transformations = []

    def add_transformation(self, func, *args, **kwargs):
        """
        Add a transformation function to the list of transformations.
        """
        self.transformations.append((func, args, kwargs))

    def apply_transformations(self, _df: DataFrame):
        """
        Apply all stored transformations to the DataFrame in order.
        """
        for func, args, kwargs in self.transformations:
            print('\nApplying transformation: {}'.format(func.__name__))
            print('Before transformation: {}'.format(_df.shape))
            _df = func(_df, *args, **kwargs)
            print('After transformation: {}'.format(_df.shape))
        return _df

    def __getstate__(self):
        """
        Called when pickling the object. Ensures all attributes are picklable.
        """
        return self.__dict__

    def __setstate__(self, state):
        """
        Called when unpickling the object. Restores the object's state.
        """
        self.__dict__.update(state)


NameError: name 'DataFrame' is not defined

# Data Cleaning

- 🧹 Perform data expectation exercise (because no details about variables were provided)
- 🧹 Remove all duplicate rows and columns (generally one of the first step in data cleaning)
- 🧹 Based on expectation, set variables to their expected data types
- 🧹 Treat empty, missing, or null data values
- 🧹 Format text to take a standard form
- 🧹 Treat typos or misrepresented entries
- 🧹 Prepare categories for categorical data

## Section helper methods

### Investigation methods

**`gather_data_expectaction`** - Designed to collect information about the expected characteristics of columns in a loan dataset. It allows time to research, gather knowlegde

In [6]:
def gather_data_expectaction(_column_list: list) -> DataFrame:
    data_type_mapping: dict = {'1': 'numerical', '2': 'text', '3': 'date', '4': 'boolean'}
    categorical_mapping: dict = {'0': False, '1': True}
    data_expectation: dict = dict()
    for column in _column_list:
        description_input = input('\n\nBrief description of a "{}" column in a loan dataset -> '.format(column))
        print('---')
        data_type_expectation = input('Data type I would expect for a "{}" column:\n1 for numerical\n2 for text\n3 for date\n4 for boolean\n -> '.format(column))
        print('---')
        is_categorical = input('Could column "{}" be categorical?:\n0 for no\n1 for yes -> '.format(column))
        print('---')

        # Default settings
        if not data_type_expectation:
            data_type_expectation = '2'

        if not is_categorical:
            is_categorical = '0'

        data_expectation[column] = {
            'description': description_input,
            'expected type': data_type_mapping.get(data_type_expectation),
            'expected categorical': categorical_mapping.get(is_categorical)
        }

    data_expectation = DataFrame(data_expectation)
    data_expectation = data_expectation.transpose()
    data_expectation = data_expectation.reset_index()
    data_expectation = data_expectation.rename(columns={'index': 'column'})

    return data_expectation

**`find_duplicate_columns_list`** - Designed to find duplicate columns

In [7]:
def find_duplicate_columns_list(_df: DataFrame, identity_perc_threshold: float) -> list[tuple]:
    number_of_rows = _df.shape[0]
    _columns_list = _df.columns.to_list()

    duplicate_set_list: list = list()

    # Iterate through all columns
    for _anchor_column in _columns_list:

        # Create a check column list which excludes the anchor column
        check_columns = [column for column in _columns_list if column != _anchor_column]

        # Iterate through all check columns
        for check_column in check_columns:

            # Test whether anchor and check column are identical
            df_check = _df[_anchor_column] == _df[check_column]
            identical_rows = df_check.sum()

            # When the proportion of number of identical rows is at least the threshold
            if (identical_rows / number_of_rows) * 100 >= identity_perc_threshold:
                duplicate_set_list.append((_anchor_column, check_column))

    return duplicate_set_list

**`find_null_columns`** - Designed to return list of null values for text and numerical variables in a dataframe 

In [8]:
def find_null_columns(_df: DataFrame) -> tuple:
    """
    Identifies numeric and text columns with null values in the DataFrame.
    """
    _numeric_columns_with_null: list = list()
    _text_columns_with_null: list = list()
    
    for _column in _df.columns:
        _df_column = _df[_column]
        
        # Check if the column is numeric
        is_numeric = is_numeric_dtype(_df_column)
        # Check if the column is text
        is_text = is_string_dtype(_df_column)
        
        # When column has null values
        if _df_column.isnull().any():
            # When column is numeric
            if is_numeric:
                _numeric_columns_with_null.append(_column)
            # When column is text
            if is_text or (str(_df_column.dtype) == 'object'):
                _text_columns_with_null.append(_column)
    
    return _numeric_columns_with_null, _text_columns_with_null

In [9]:
def get_numerical_columns(_df: DataFrame) -> list():
    _df_columns = _df.columns.to_list()

    _numerical_cols:list = list()
    # Iterate through all columns
    for column in _df_columns:

        # When data type is numerical
        if is_numeric_dtype(_df[column]):
            _numerical_cols.append(column)

    return _numerical_cols

In [10]:
def get_text_columns(_df: DataFrame, category_perc_threshold: float) -> list:
    _df_rows = _df.shape[0]
    max_unique_values = (category_perc_threshold / 100) * _df_rows
    text_cols:list = list()

    # Iterate through all columns
    for col in _df.columns:

        # When type is object
        column_dtype = _df[col].dtype
        if column_dtype == 'object':
            unique_values = _df[col].nunique()

            # When unique values are at least the threshold
            if unique_values > max_unique_values:
                text_cols.append(col)

    return text_cols

In [11]:
def get_categorical_columns(_df: DataFrame, category_perc_threshold: float) -> list:
    _df_rows = _df.shape[0]
    max_unique_values = (category_perc_threshold / 100) * _df_rows
    categorical_cols:list = list()

    # Iterate through all columns
    for col in _df.columns:

        # When type is object
        column_dtype = _df[col].dtype
        if column_dtype == 'object':
            unique_values = _df[col].nunique()

            # When unique values are at most the threshold
            if unique_values <= max_unique_values:
                categorical_cols.append(col)

        # When type is categorical
        if isinstance(column_dtype, CategoricalDtype):
            categorical_cols.append(col)

    return categorical_cols

In [12]:
def find_similar_values(_df: DataFrame, _column: str, threshold: int = 80) -> dict:
    unique_values = _df[_column].unique()
    similar_values_list: list = list()

    for value in unique_values:
        # Extract similar values to the current value
        matches = process.extract(value, unique_values, limit=None)
        similar_values = [match[0] for match in matches if match[1] >= threshold and match[0] != value]

        if similar_values:
            similar_values.append(value)
            similar_values_list.append({value: similar_values})

    return {_column: similar_values_list}

### Raw data cleaning methods

In [13]:
def replace_empty_or_blank_values(_df: DataFrame):
    """
    Replaces empty spaces with NaN values in the DataFrame
    """
    _columns_list = _df.columns.to_list()
    for _column in _columns_list:
        _df[_column] = _df[_column].replace(r'^\s*$', nan_value, regex=True)
        
    return _df

In [14]:
# Function created to use as input in Transformation module
def drop_dataframe_columns(_df: DataFrame, drop_column_list: list) -> DataFrame:
    _df = _df.drop(columns=drop_column_list)
    return _df

In [15]:
def convert_from_text_to_date(_df: DataFrame, _columns_list: list) -> DataFrame:
    for _column in _columns_list:
        _df[_column] = to_datetime(_df[_column])
        
    return _df

In [16]:
def convert_from_text_to_numeric(_df: DataFrame, _columns_list: list) -> DataFrame:
    for _column in _columns_list:
        _df[_column] = to_numeric(_df[_column], errors='coerce')
        
    return _df

In [17]:
def convert_to_text(_df: DataFrame, _columns_list: list) -> DataFrame:
    for _column in _columns_list:
        _df[_column] = _df[_column].astype(str)
        
    return _df

In [18]:
def treat_missing_by_mode_imputation(_df: DataFrame, _columns_list: list):
    for column in _columns_list:
        replace_value = _df[column].mode()
        _df[column] = _df[column].fillna(replace_value[0])

    return _df

In [19]:
def treat_missing_by_mean_imputation(_df: DataFrame, _columns_list: list):
    for column in _columns_list:
        replace_value = _df[column].mean()
        _df[column] = _df[column].fillna(replace_value)

    return _df

In [20]:
def clean_text_columns(_df: DataFrame, column_names: list) -> DataFrame:
    for col in column_names:
        _df[col] = _df[col].str.strip().str.lower()
    return _df

In [21]:
def make_value_corrections(_df: DataFrame, instructions: list[dict]) -> DataFrame:
    for instruction in instructions:
        column = instruction.get('column')
        value = instruction.get('value')
        to_correct = instruction.get('to correct')

        _df[column] = _df[column].replace(to_correct, value)

    return _df

## Data inspection

### Read data

In [22]:
raw_data = read_csv('{}/data_science_competition_2024.csv'.format(DATA_DIR))
raw_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/data_science_competition_2024.csv'

In [ ]:
raw_data.shape

### Gather data expectations

In [ ]:
df_column_list = raw_data.columns.to_list()
df_column_list

In [ ]:
# expectations_df = gather_data_expectaction(df_column_list)

In [ ]:
expectations_df = DataFrame(
[{'column': 'Unnamed: 0',
  'description': '',
  'expected type': 'text',
  'expected categorical': False},
 {'column': 'loan_id',
  'description': '',
  'expected type': 'text',
  'expected categorical': False},
 {'column': 'gender',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'disbursemet_date',
  'description': '',
  'expected type': 'date',
  'expected categorical': False},
 {'column': 'currency',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'country',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'sex',
  'description': 'may be a duplicate of gender',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'is_employed',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'job',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'location',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'loan_amount',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'number_of_defaults',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': True},
 {'column': 'outstanding_balance',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'interest_rate',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'age',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'number_of_defaults.1',
  'description': 'may be a duplicate of number of defaults',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'remaining term',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'salary',
  'description': '',
  'expected type': 'numerical',
  'expected categorical': False},
 {'column': 'marital_status',
  'description': '',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'age.1',
  'description': 'could be a duplicate of age',
  'expected type': 'text',
  'expected categorical': True},
 {'column': 'Loan Status',
  'description': '',
  'expected type': 'text',
  'expected categorical': True}])

In [ ]:
expectations_df

### Prepare inference data

In [ ]:
target_variable = 'Loan Status'
raw_data_columns_list = df_column_list.copy()
target_variable_column_index = df_column_list.index(target_variable)
raw_data_columns_list.pop(target_variable_column_index)
raw_data_columns_list

In [ ]:
raw_X = raw_data[raw_data_columns_list]
raw_y = raw_data[target_variable]

# Perform stratified split
_, X_inference, _, y_inference = train_test_split(raw_X, raw_y, test_size=0.0025, stratify=raw_y, random_state=42)

In [ ]:
X_inference.shape

In [ ]:
X_inference.to_csv('{}/inference_data.csv'.format(DATA_DIR), index=False)

## Data cleaning process

In [ ]:
clean_df = raw_data.copy()

In [ ]:
clean_df.shape

In [ ]:
# Initialize the transformation module with the DataFrame
trans_module = TransformationModule()

### Duplicate rows

In [ ]:
clean_df = clean_df.drop_duplicates()
clean_df.shape

### Duplicate columns

In [ ]:
duplicate_columns_list = find_duplicate_columns_list(clean_df, 99)
duplicate_columns_list

In [ ]:
duplicates_drop_list: list = list()
request_check_list: list = list()
for duplicate_set in duplicate_columns_list:
    set_part_1, set_part_2 = duplicate_set
    if (set_part_1 not in request_check_list) and (set_part_2 not in request_check_list):
        set_response = input('Keep "{}" and drop "{}"?: '.format(set_part_1, set_part_2))

        if int(set_response) == 1:
            duplicates_drop_list.append(set_part_2)
            request_check_list.extend([set_part_1, set_part_2])

In [ ]:
duplicates_drop_list

In [ ]:
# Perform transformation
clean_df = drop_dataframe_columns(clean_df, duplicates_drop_list)
# Store transformation step
trans_module.add_transformation(drop_dataframe_columns, drop_column_list=duplicates_drop_list)
clean_df.shape

### Data type cleaning

In [ ]:
clean_df.dtypes

In [ ]:
date_in_text_columns = ['disbursemet_date']
clean_df = convert_from_text_to_date(clean_df, date_in_text_columns)

# Add transformation step
trans_module.add_transformation(convert_from_text_to_date, _columns_list=date_in_text_columns)

In [ ]:
numerical_in_text_columns = ['remaining term']
clean_df = convert_from_text_to_numeric(clean_df, numerical_in_text_columns)

# Add transformation step
trans_module.add_transformation(convert_from_text_to_numeric, _columns_list=numerical_in_text_columns)

In [ ]:
should_be_categorical_columns_list = ['is_employed', 'number_of_defaults']
clean_df = convert_to_text(clean_df, should_be_categorical_columns_list)

# Add transformation step
trans_module.add_transformation(convert_to_text, _columns_list=should_be_categorical_columns_list)

### Null and empty cell value treatment

In [ ]:
clean_df = replace_empty_or_blank_values(clean_df)

# Add transformation step
trans_module.add_transformation(replace_empty_or_blank_values)

In [ ]:
numeric_columns_with_null, text_columns_with_null = find_null_columns(clean_df)
print(numeric_columns_with_null)
print(text_columns_with_null)

In [ ]:
clean_df = treat_missing_by_mode_imputation(clean_df, text_columns_with_null)

# Add transformation step
trans_module.add_transformation(treat_missing_by_mode_imputation, _columns_list=text_columns_with_null)

In [ ]:
clean_df = treat_missing_by_mean_imputation(clean_df, numeric_columns_with_null)

# Add transformation step
trans_module.add_transformation(treat_missing_by_mean_imputation, _columns_list=numeric_columns_with_null)

In [ ]:
numeric_columns_with_null, text_columns_with_null = find_null_columns(clean_df)
print(numeric_columns_with_null)
print(text_columns_with_null)

### Text column formatting

In [ ]:
text_type_columns_list:list = list()
for column in clean_df.columns.to_list():
    if is_string_dtype(clean_df[column]):
        text_type_columns_list.append(column)
text_type_columns_list

In [ ]:
clean_df = clean_text_columns(clean_df, text_type_columns_list)

# Add transformation step
text_type_columns_list.remove(target_variable)
trans_module.add_transformation(clean_text_columns, column_names=text_type_columns_list)

### Category value correction

In [ ]:
categorical_columns_list = get_categorical_columns(clean_df, 1)
categorical_columns_list

In [ ]:
value_correction_column_list: list = list()

# Iterate through all categorical
for column in categorical_columns_list:
    correction_list = find_similar_values(clean_df, column)

    # When similar values have been detected
    if correction_list.get(column):
        value_correction_column_list.append(correction_list)

value_correction_column_list

In [ ]:
# Iterate through list of correction columns
corrections_list: list = list()
for column_corrections in value_correction_column_list:
    column_name = list(column_corrections.keys())[0]
    print('\nUnder column "{}":'.format(column_name))

    # Iterate through similarity suggestions
    for correction in column_corrections:
        suggestions = column_corrections.get(correction)
        skip_list: list = list()
        # Iterate through each phrase
        for suggestion in suggestions:
            suggestion_key = list(suggestion.keys())[0]
            suggestion_list = suggestion.get(suggestion_key)

            # Check if value has been mapped
            if suggestion_key not in skip_list:
                correct_response = input("Allow '{}' to represent: {} -> ".format(suggestion_key, suggestion_list))

                # When user has agreed to correct
                if int(correct_response) == 1:
                    corrections_list.append({'column': column_name, 'value': suggestion_key, 'to correct': suggestion_list})

                    # Add the list as already mapped in skipped list
                    skip_list.extend(suggestion_list)


In [ ]:
corrections_list

In [ ]:
clean_df = make_value_corrections(clean_df, corrections_list)

# Add transformation step
trans_module.add_transformation(make_value_corrections, instructions=corrections_list)

In [ ]:
value_correction_column_list: list = list()

# Iterate through all categorical
for column in categorical_columns_list:
    correction_list = find_similar_values(clean_df, column)

    # When similar values have been detected
    if correction_list.get(column):
        value_correction_column_list.append(correction_list)

value_correction_column_list

In [ ]:
clean_df.shape

# Exploratory data analysis

- 📊 Understanding and analyzing the distribution, range, and potential issues of the target variables
- 📊 Analysis of distibution, central tendency, and spread of the numerical variables, including the identification of outliers, skewness, or kurtosis.
- 📊 Identification and analysis of text-based variables
- 📊 Analysis of the distribution and frequency of categories
- 📊 Identification and analysis of date or time-based variables and extraction of relevant features and exploring the temporal patterns

## Section helper methods

In [ ]:
def display_numerical_features_visualization(_df: DataFrame, _column_list: list, viz_name: str,  num_of_cols_per_grid: int=3):

    # Determine the number of rows needed based on the total number of columns
    n_columns = len(_column_list)
    grid_rows = (n_columns + (num_of_cols_per_grid - 1)) // num_of_cols_per_grid  # Round up the number of rows

    # Create the figure and subplots
    fig, axs = plt.subplots(grid_rows, num_of_cols_per_grid, figsize=(12, 4 * grid_rows))

    # Flatten the axes array to make it easier to access individual subplots
    axs = axs.flatten()

    # Plot the histograms in the subplots
    for i, column_name in enumerate(_df[_column_list].columns[:n_columns]):
        sns.histplot(data=_df, x=column_name, color="skyblue",hue=target_variable, ax=axs[i])
        axs[i].set_title(column_name)

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    plt.savefig(viz_name)
    plt.close()

In [ ]:
def display_categorical_features_visualization(_df: DataFrame, _column_list: list, viz_name: str, num_of_cols_per_grid: int=3):

    # Determine the number of rows needed based on the total number of columns
    n_columns = len(_column_list)
    grid_rows = (n_columns + (num_of_cols_per_grid - 1)) // num_of_cols_per_grid  # Round up the number of rows

    # Create the figure and subplots
    fig, axs = plt.subplots(grid_rows, num_of_cols_per_grid, figsize=(12, 4 * grid_rows))

    # Flatten the axes array to make it easier to access individual subplots
    axs = axs.flatten()

    # Plot the countplots in the subplots
    for i, column_name in enumerate(_df[_column_list].columns[:n_columns]):
        sns.countplot(x=column_name, data=clean_df, hue=target_variable, ax=axs[i])
        axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=60)
        axs[i].set_title(column_name)

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    plt.savefig(viz_name)
    plt.close()

In [ ]:
def display_boxplot_visualization(_df: DataFrame, _column_list: list, viz_name: str, num_of_cols_per_grid: int=3):

    # Determine the number of rows needed based on the total number of columns
    n_columns = len(_column_list)
    grid_rows = (n_columns + (num_of_cols_per_grid - 1)) // num_of_cols_per_grid  # Round up the number of rows

    # Create the figure and subplots
    fig, axs = plt.subplots(grid_rows, num_of_cols_per_grid, figsize=(12, 4 * grid_rows))

    # Flatten the axes array to make it easier to access individual subplots
    axs = axs.flatten()

    # Plot the countplots in the subplots
    for i, column_name in enumerate(_df[_column_list].columns[:n_columns]):
        sns.boxplot(y=_df[column_name], ax=axs[i])
        axs[i].set_title(column_name)

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    plt.savefig(viz_name)
    plt.close()

## Target

In [ ]:
# target_variable
Y = clean_df[target_variable]
Y.head()

In [ ]:
# Independent variables
X = clean_df.drop(columns=[target_variable])
X.head()

In [ ]:
# Explore Target distribution
sns.catplot(x=target_variable, kind="count", data=clean_df)
_viz_name = 'eda/target_distribution.png'
plt.savefig(_viz_name)

# Display the image
# Image(filename=_viz_name)

### Numerical variables

In [ ]:
numerical_columns_list = get_numerical_columns(X)
numerical_columns_list

In [ ]:
_viz_name = 'eda/numerical_distribution.png'
display_numerical_features_visualization(clean_df, numerical_columns_list, _viz_name)
# Display the image
Image(filename=_viz_name)

### Text variables

In [ ]:
text_columns_list = get_text_columns(X, 1)
text_columns_list

### Categorical variables

In [ ]:
categorical_columns_list = get_categorical_columns(X, 1)
categorical_columns_list

In [ ]:
_viz_name = 'eda/categorical_distribution.png'
display_categorical_features_visualization(clean_df, categorical_columns_list, _viz_name)
# Display the image
Image(filename=_viz_name)

### Date or time variables

In [ ]:
counts = clean_df.groupby([DatetimeIndex(clean_df['disbursemet_date']).month, target_variable])['disbursemet_date'].count()
counts.unstack(fill_value=0).plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Loan disbursed Count by Loan Status')
_viz_name = 'eda/date_distribution.png'
plt.savefig(_viz_name)
# Display the image
Image(filename=_viz_name)

# Feature engineering

- 🛠️ Exploring the dataset to identify potential features for engineering
- 🛠️ Creation of new features from the existing data to improve model performance
- 🛠️ Applying an iterative process of creating, evaluating, and selecting relevant features
- 🛠️Identifying and removing features that do not provide additional information to the model
- 🛠️ Handling features with a large number of unique categories
- 🛠️ Ensuring numerical features have a similar scale to avoid biased models
- 🛠️ Using visualizations to identify and handle outliers in the data
- 🛠️ Applying statistical techniques to identify and handle outliers.
- 🛠️ Transforming skewed numerical features to improve their distribution
- 🛠️ Limiting the influence of extreme values by capping them at a specified percentile
- 🛠️Rescaling numerical features to have a mean of 0 and a standard deviation of 1
- 🛠️ Continuous grouping of numerical features into discrete bins
- 🛠️ Converting categorical features into a format that can be used by machine learning models
- 🛠️ Choosing the most informative features to improve model performance and reduce complexity

## Section helper methods

### Investigation methods

In [ ]:
def get_high_cardinality_columns(_df: DataFrame, category_perc_threshold: float) -> list:
    _df_rows = _df.shape[0]
    max_unique_values = (category_perc_threshold / 100) * _df_rows
    high_cardinality_cols:list = list()

    # Iterate through all columns
    _categorical_cols: list = list()
    for col in _df.columns:

        # When type is object
        column_dtype = _df[col].dtype
        if column_dtype == 'object':
            unique_values = _df[col].nunique()

            # When unique values are at most the threshold
            if unique_values <= max_unique_values:
                _categorical_cols.append(col)

        # When type is categorical
        if isinstance(column_dtype, CategoricalDtype):
            _categorical_cols.append(col)

    return _categorical_cols

### Engineering methods

In [ ]:
def bin_data_with_outliers(_df: DataFrame, column_name: str, _bin_details: dict) -> DataFrame:
    # Unpack variables
    min_category_cap = _bin_details.get('min_category_cap')
    max_category_cap = _bin_details.get('max_category_cap')
    num_bins = _bin_details.get('num_bins')
    
    # Compute the bin edges
    bins = linspace(min_category_cap, max_category_cap, num_bins - 1)

    # Ensure max_category_cap is not duplicated in the bins
    if max_category_cap not in bins:
        bins = concatenate((bins, [max_category_cap]))
    else:
        bins = bins.tolist()
        bins[-1] = max_category_cap

    edges = [-np_inf] + list(bins) + [np_inf]

    # Generate bin labels
    bin_labels = []
    for i in range(len(edges) - 1):
        if i == 0:
            bin_label = f"below {edges[i+1]}"
        elif i == len(edges) - 2:
            bin_label = f"above {edges[i]}"
        else:
            bin_label = f"{edges[i]} to {edges[i+1]}"
        bin_labels.append(bin_label)

    # Bin the data, including outliers in the first and last bins
    binned_data = cut(_df[column_name], bins=edges, labels=bin_labels, include_lowest=True)

    # Add the binned data as a new column
    binned_column_name = "binned_{}".format(column_name)
    _df[binned_column_name] = binned_data

    return _df

In [ ]:
def log_transform_columns(_df: DataFrame, _columns_list_original: list, _columns_list_transformed: list) -> DataFrame:
    _df[_columns_list_transformed] = _df[_columns_list_original].applymap(lambda x: log(x + 1))
    return _df

In [ ]:
def winsorize_columns(_df: DataFrame, _columns_list: list, outlier_threshold: tuple) -> DataFrame:
    for _column in _columns_list:
        _df[_column] = winsorize(_df[_column], outlier_threshold)

    return _df

In [ ]:
def normalize_by_standard_scaling(_df: DataFrame, _columns_list: list) -> DataFrame:
    scaler = StandardScaler()
    _df[_columns_list] = scaler.fit_transform(_df[_columns_list])

    return _df

In [ ]:
def one_hot_encode_columns(_df: DataFrame, _columns_list: list) -> DataFrame:
    print('Original dimensions are {}'.format(_df.shape))
    
    # Initialize the OneHotEncoder
    encoder = OneHotEncoder(handle_unknown='ignore')
    
    # Fit and transform the categorical columns
    encoded_columns = encoder.fit_transform(_df[_columns_list])
    
    # Create a DataFrame from the one-hot encoded columns
    column_names = encoder.get_feature_names_out(_columns_list)
    
    # Convert to dense array if it's sparse
    if hasattr(encoded_columns, 'toarray'):
        encoded_columns = encoded_columns.toarray()
    
    encoded_df = DataFrame(encoded_columns, columns=column_names, index=_df.index)
    print('Encoded columns have {} dimensions'.format(encoded_df.shape))
    
    # Concatenate the one-hot encoded columns with the original DataFrame
    print('Removing {} existing columns'.format(len(_columns_list)))
    
    _df = concat([_df.drop(columns=_columns_list), encoded_df], axis=1)
    print('Merged columns have {} dimensions'.format(_df.shape))
    
    return _df

## Feature engineering process

In [ ]:
X_features = X.copy()

In [ ]:
X_features.head()

In [ ]:
X_features.shape

### Redundant features
- Prove redundancy by EDA
- Prove redundancy by unique value

In [ ]:
redundant_features_list = ['currency', 'country', 'disbursemet_date']
X_features = drop_dataframe_columns(X_features, redundant_features_list)

# Add transformation step
trans_module.add_transformation(drop_dataframe_columns, drop_column_list=redundant_features_list)
X_features.shape

### High cardinality features
- Prove by uniform distribution

In [ ]:
high_cardinality_features_list = ['Unnamed: 0', 'loan_id']
X_features = drop_dataframe_columns(X_features, high_cardinality_features_list)

# Add transformation step
trans_module.add_transformation(drop_dataframe_columns, drop_column_list=high_cardinality_features_list)
X_features.shape

### Feature scaling for numerical features

In [ ]:
numerical_columns_list = get_numerical_columns(X_features)
numerical_columns_list

#### Outlier detection visual method

In [ ]:
_viz_name = 'eda/outlier.png'
display_boxplot_visualization(X_features, numerical_columns_list, _viz_name)
# Display the image
Image(filename=_viz_name)

#### Outlier detection statistical method

In [ ]:
number_of_rows = X_features.shape[0]
outliers_list_details: list = list()
for numerical_column in numerical_columns_list:
    # Calculate Z-scores
    z_scores = stats.zscore(X_features[numerical_column])
    outliers_upper = X_features[numerical_column][z_scores > 3]
    outliers_lower = X_features[numerical_column][z_scores < -3]

    # Find the value corresponding to z = 3
    value_at_z3 = quantile(X_features[numerical_column], stats.norm.cdf(3))

    # Find the value corresponding to z = -3
    value_at_nz3 = quantile(X_features[numerical_column], stats.norm.cdf(-3))

    outlier_results = {
        'column': numerical_column,
        'lower_outliers_count': outliers_lower.count(),
        'upper_outliers_count': outliers_upper.count(),
        'min_value': min(X_features[numerical_column]),
        'max_value': max(X_features[numerical_column]),
        'lower_value_limit': value_at_nz3,
        'upper_value_limit': value_at_z3
    }
    outliers_list_details.append(outlier_results)

outliers_details_df = DataFrame(outliers_list_details)
outliers_details_df

In [ ]:
transform_to_log_columns_list = ['loan_amount', 'outstanding_balance', 'salary']
transform_to_log_binned_columns_list = ['interest_rate', 'age', 'remaining term']

#### Log transformation for skewed features

In [ ]:
transformed_log_columns_list = ['log_{}'.format(column) for column in transform_to_log_columns_list]
transformed_log_columns_list

In [ ]:
X_features = log_transform_columns(X_features, transform_to_log_columns_list, transformed_log_columns_list)

# Add transformation step
trans_module.add_transformation(log_transform_columns, _columns_list_original=transform_to_log_columns_list, _columns_list_transformed=transformed_log_columns_list)
X_features.shape

#### Feature winsorization

In [ ]:
X_features = winsorize_columns(X_features, transformed_log_columns_list, (0.05, 0.05))

# Add transformation step
trans_module.add_transformation(winsorize_columns, _columns_list=transformed_log_columns_list, outlier_threshold=(0.05, 0.05))
X_features.shape

In [ ]:
before_and_after_columns_list = transform_to_log_columns_list + transformed_log_columns_list
_viz_name = 'eda/outlier_before_and_after_log_and_winsorization.png'
display_boxplot_visualization(X_features, before_and_after_columns_list, _viz_name)
# Display the image
Image(filename=_viz_name)

In [ ]:
X_features = drop_dataframe_columns(X_features, transform_to_log_columns_list)

# Add transformation step
trans_module.add_transformation(drop_dataframe_columns, drop_column_list=transform_to_log_columns_list)
X_features.shape

#### Normalize scaling

In [ ]:
X_features = normalize_by_standard_scaling(X_features, transformed_log_columns_list)

# Add transformation step
trans_module.add_transformation(normalize_by_standard_scaling, _columns_list=transformed_log_columns_list)
X_features.shape

### Feature binning for numerical features

In [ ]:
transform_to_log_binned_columns_list

In [ ]:
# Assuming you have a dataframe 'df' and a list of names 'name_list'
outliers_details_df_binning_columns = outliers_details_df[outliers_details_df['column'].isin(transform_to_log_binned_columns_list)]
outliers_details_df_binning_columns

In [ ]:
_binning_column = 'interest_rate'
bin_details = {'num_bins': 6, 'min_category_cap': 0.15, 'max_category_cap': 0.25}
X_features = bin_data_with_outliers(X_features, _binning_column, bin_details)

# Add transformation step
trans_module.add_transformation(bin_data_with_outliers, column_name=_binning_column, _bin_details=bin_details)

# Display results
bin_stats = X_features.groupby('binned_{}'.format(_binning_column))[_binning_column].agg(['min', 'max', 'count']).reset_index()
print(X_features.shape, '\n____________')
bin_stats

In [ ]:
_binning_column = 'age'
bin_details = {'num_bins': 9, 'min_category_cap': 25, 'max_category_cap': 60}
X_features = bin_data_with_outliers(X_features, _binning_column, bin_details)

# Add transformation step
trans_module.add_transformation(bin_data_with_outliers, column_name=_binning_column, _bin_details=bin_details)

# Display results
bin_stats = X_features.groupby('binned_{}'.format(_binning_column))[_binning_column].agg(['min', 'max', 'count']).reset_index()
print(X_features.shape, '\n____________')
bin_stats

In [ ]:
_binning_column = 'remaining term'
bin_details = {'num_bins': 10, 'min_category_cap': 20, 'max_category_cap': 100}
X_features = bin_data_with_outliers(X_features, _binning_column, bin_details)

# Add transformation step
trans_module.add_transformation(bin_data_with_outliers, column_name=_binning_column, _bin_details=bin_details)

# Display results
bin_stats = X_features.groupby('binned_{}'.format(_binning_column))[_binning_column].agg(['min', 'max', 'count']).reset_index()
print(X_features.shape, '\n____________')
bin_stats

In [ ]:
X_features = drop_dataframe_columns(X_features, transform_to_log_binned_columns_list)

# Add transformation step
trans_module.add_transformation(drop_dataframe_columns, drop_column_list=transform_to_log_binned_columns_list)
X_features.shape

### Feature encoding for categorical features

In [ ]:
# Encode the Y feature
target_variable_mapping = {'defaulted': 1, 'did not default': 0}
y = Y.replace(target_variable_mapping)
y.value_counts()

In [ ]:
categorical_columns_list = get_categorical_columns(X_features, 1)
categorical_columns_list

In [ ]:
X_features = one_hot_encode_columns(X_features, categorical_columns_list)

# Add transformation step
trans_module.add_transformation(one_hot_encode_columns, _columns_list=categorical_columns_list)
X_features.shape

In [ ]:
X_features.dtypes

### Feature selection

In [ ]:
def calculate_vif(_df: DataFrame):
    """
    Calculate Variance Inflation Factor (VIF) for each feature in the DataFrame.
    """
    vif_data = DataFrame()
    vif_data["feature"] = _df.columns
    vif_data["VIF"] = [variance_inflation_factor(_df.values, i) for i in range(len(_df.columns))]
    return vif_data

In [ ]:
def remove_multicollinearity(_df: DataFrame, threshold: float = 5.0):
    """
    Remove multicollinearity from the DataFrame by eliminating features with high VIF.
    """
    new_df = _df.copy()
    while True:
        vif_data = calculate_vif(new_df)
        max_vif = vif_data["VIF"].max()

        if max_vif > threshold:
            feature_to_remove = vif_data.loc[vif_data["VIF"].idxmax(), "feature"]
            print(f"Removing feature '{feature_to_remove}' with VIF: {max_vif}")
            new_df = new_df.drop(columns=[feature_to_remove])
        else:
            break

    return new_df

# Data transformation summary

- Identification and exploring suitable data transformation techniques
- Evaluating the impact of different transformations on model performance
- Storing the selected transformations for consistent application
- Preperation of the dataset, including handling missing values, encoding features, and scaling
- Dividing the dataset to assess the model's performance on unseen data
- Addressing class imbalance issues using appropriate balancing techniques

### Section helper methods

In [ ]:
def dataset_similarity_report(_df1: DataFrame, _df2: DataFrame):
    # Column test
    df1_columns = _df1.columns.to_list()
    df2_columns = _df2.columns.to_list()

    # When column names are identical
    if df1_columns == df2_columns:
        print('Dataset columns identical')

        non_identical_columns: list = list()
        for _column in df1_columns:

            # When row items are identical
            identity_check = _df1[_column].to_list() == _df2[_column].to_list()
            if not identity_check:
                non_identical_columns.append(_column)

        # When columns with unidentical rows have been identified
        if non_identical_columns:
            print(non_identical_columns, ': are not row identical')

        # When all records are identical
        else:
            print('All records for each column are identical')
    
    # When columns names do not match
    else:
        items_in_list1_not_in_list2 = set(df1_columns) - set(df2_columns)
        items_in_list2_not_in_list1 = set(df2_columns) - set(df1_columns)
        print("Columns in dataset 1 that are not in dataset 2:", items_in_list1_not_in_list2)
        print("Columns in dataset 2 that are not in dataset 1:", items_in_list2_not_in_list1)

In [ ]:
def save_transformations(transformation_module, _module_name: str='loan_data_transformation'):
    # Check for any transformations in directory
    module_list = listdir(TRANSFORMATIONS_DIR)

    # When module list exists
    check_name_count: int = 0
    if module_list:
        check_name_list = [module_name_ for module_name_ in module_list if _module_name in module_name_]
        check_name_count = len(check_name_list)

    # Indicate the nth of the coming report
    check_name_count += 1
    module_file_name = '{}/{}_{}.pkl'.format(TRANSFORMATIONS_DIR, _module_name, check_name_count)
    
    # Save the TransformationModule object
    with open(module_file_name, 'wb') as f:
        dill.dump(transformation_module, f)

    # When report has been generated
    if exists(module_file_name):
        print('Module: {} successfully saved!'.format(module_file_name))

    else:
        print('No generated module!')

### Check transformations

In [ ]:
_i = 1
for _transformation in trans_module.transformations:
    transformation_function = _transformation[0]
    print('\nStep {}:'.format(_i))
    print(transformation_function.__name__, _transformation[2])
    _i += 1

### Test transformations

In [ ]:
transformation_test_df = raw_X.copy()
transformation_test_df.shape

In [ ]:
df_transformed = trans_module.apply_transformations(transformation_test_df)

In [ ]:
dataset_similarity_report(X_features, df_transformed)

### Save transformations for inference

In [ ]:
save_transformations(trans_module)

# Data pre-processing

### Split to train and test

In [ ]:
# Split the data into train and test sets
X_feature_train, X_test, y_feature_train, y_test = train_test_split(X_features, y, test_size=0.3, random_state=42)

In [ ]:
print('X_feature_train: ', X_feature_train.shape, '; y_feature_train: ', y_feature_train.shape)
print('X_test: ', X_test.shape, '; y_test: ', y_test.shape)

In [ ]:
print('Original training set class distribution:', Counter(y_feature_train))

### Apply Balancing technique

In [ ]:
# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_feature_train, y_feature_train)
print('X_train: ', X_train.shape,' ; y_train: ', y_train.shape)

In [ ]:
# Check the class distribution in the resampled training set
print('Resampled training set class distribution:', Counter(y_train))

# Model building

- Choosing an appropriate classification algorithm and configure its initial settings
- Optimizing the model's hyperparameters through a systematic process of cross-validation

## Section helper methods

In [ ]:
def compile_cv_results(desired_run_id: int):
    # Initialize an empty list to store the results
    _cv_results: list = list()
    _best_params_dict: dict = dict()
    # Iterate through the files in the directory
    for filename in listdir(CROSS_VALIDATION_DIR):
        if filename.endswith('.pkl'):
            # Extract the run_id and setup_name from the filename
            _file_details = filename.split('_')
            _run_id, _setup_name = _file_details[0], _file_details[1]
            _run_id = int(_run_id)
    
            # Check if the run_id matches the one you're interested in
            if _run_id == desired_run_id:
                # Load the pickled data
                with open('{}/{}'.format(CROSS_VALIDATION_DIR, filename), 'rb') as f:
                    _data = pickle.load(f)
                f.close()
    
                # Add the results to the list
                _cv_results.append({**_data['result'], 'setup_name': _setup_name})

                # Add params to dict
                _best_params_dict[_setup_name] = _data.get('params')

    # Create the DataFrame
    _cv_results_df = DataFrame(_cv_results)

    return _cv_results_df, _best_params_dict

The weights in the scoring function (0.3 for ROC AUC, 0.2 each for precision, recall, and F1, and 0.1 for accuracy) are chosen to prioritize ROC AUC while giving equal importance to precision, recall, and F1, and slightly less importance to accuracy.

In [ ]:
def score_model(model):
    return (
        0.3 * model['roc_auc'] +
        0.2 * model['precision'] +
        0.2 * model['recall'] +
        0.2 * model['f1'] +
        0.1 * model['accuracy']
    )

In [ ]:
def get_best_models(results_df: DataFrame, n=5) -> DataFrame:
    
    # Calculate the combined score
    results_df['score'] = results_df.apply(score_model, axis=1)
    
    # Sort by score in descending order and select top n
    _best_models_df = results_df.sort_values('score', ascending=False).head(n)
    
    return _best_models_df

## Classification algorithm setup

In [ ]:
# Classifier instances
lr = LogisticRegression()
lda = LinearDiscriminantAnalysis()
dt = DecisionTreeClassifier()
et = ExtraTreesClassifier()
gbc = GradientBoostingClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier(force_row_wise=true)
svc = SVC(probability=True)
knn = KNeighborsClassifier()
gnb = GaussianNB()

In [ ]:
# Hyperparameter dictionaries
lr_param_dist = {
    'C': stats.uniform(0.1, 10),
    'penalty': ['l2', 'l1', 'elasticnet'],
    'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4, 1e-5]
}

lda_param_dist = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto'] + list(stats.uniform(0, 1).rvs(5))
}

dt_param_dist = {
    'max_depth': stats.randint(3, 20),
    'min_samples_split': stats.randint(2, 11),
    'min_samples_leaf': stats.randint(1, 11)
}

et_param_dist = {
    'n_estimators': stats.randint(50, 300),
    'max_depth': stats.randint(3, 20),
    'min_samples_split': stats.randint(2, 11),
    'min_samples_leaf': stats.randint(1, 11)
}

gbc_param_dist = {
    'n_estimators': stats.randint(50, 300),
    'learning_rate': stats.uniform(0.01, 0.3),
    'max_depth': stats.randint(3, 20)
}

rf_param_dist = {
    'n_estimators': stats.randint(50, 300),
    'max_features': ['sqrt', 'log2', None],
    'max_depth': stats.randint(3, 20),
    'min_samples_split': stats.randint(2, 11),
    'min_samples_leaf': stats.randint(1, 11)
}

xgb_param_dist = {
    'max_depth': stats.randint(3, 20),
    'learning_rate': stats.uniform(0.01, 0.3),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators': stats.randint(50, 300)
}

lgbm_param_dist = {
    'num_leaves': stats.randint(20, 100),
    'learning_rate': stats.uniform(0.01, 0.3),
    'n_estimators': stats.randint(50, 300)
}

svc_param_dist = {
    'C': stats.uniform(0.1, 10),
    'kernel': ['rbf', 'linear', 'poly'],
    'gamma': ['scale', 'auto'] + list(stats.uniform(0.01, 1).rvs(3))
}

knn_param_dist = {
    'n_neighbors': stats.randint(3, 20),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

gnb_param_dist = {
    'var_smoothing': stats.uniform(1e-10, 1e-8)
}

In [ ]:
modelling_setup = [
    {'name': 'xgb', 'classifier': xgb, 'params': xgb_param_dist},
    {'name': 'gnb', 'classifier': gnb, 'params': gnb_param_dist},
    {'name': 'lr', 'classifier': lr, 'params': lr_param_dist},
    {'name': 'lda', 'classifier': lda, 'params': lda_param_dist},
    {'name': 'knn', 'classifier': knn, 'params': knn_param_dist},
    {'name': 'svc', 'classifier': svc, 'params': svc_param_dist},
    {'name': 'dt', 'classifier': dt, 'params': dt_param_dist},
    {'name': 'rf', 'classifier': rf, 'params': rf_param_dist},
    {'name': 'et', 'classifier': et, 'params': et_param_dist},
    {'name': 'gbc', 'classifier': gbc, 'params': gbc_param_dist},
    {'name': 'lgbm', 'classifier': lgbm, 'params': lgbm_param_dist}
]

## Hyper parameter tuning using cross-validation

In [ ]:
# Define scoring metrics
scoring = {
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

# Time limit for each model
time_limit = timedelta(minutes=45)

# Run ID represents set of parameters set for the Randomsearch. Files with the different run_ids should not be compared

run_id = 3
# cv=5,
# n_iter=10,  # high number for better exploration
# refit='roc_auc',
# random_state=5,
# n_jobs=-1,  # Use all available cores
# verbose=2,
# return_train_score=True

# run_id = 1
# cv=5,
# n_iter=15,  # high number for better exploration
# refit='roc_auc',
# random_state=42,
# n_jobs=-1,  # Use all available cores
# verbose=2,
# return_train_score=True

for _setup in modelling_setup:
    
    setup_name = _setup['name']
    if setup_name not in ['dt', 'et', 'lgbm']:
        continue
    
    print('\n{}-------------'.format(setup_name))
    _clf = _setup['classifier']
    _param_dist = _setup['params']
    
    start_time = datetime.now()
    
    random_search = RandomizedSearchCV(
        estimator=_clf,
        param_distributions=_param_dist,
        cv=5,  # low number for faster execution
        scoring=scoring,
        n_iter=10,  # high number for better exploration
        refit='roc_auc',
        random_state=5,
        # n_jobs=-1,  # Use all available cores
        verbose=2,
        # return_train_score=True
    )
    
    try:
        # Use a separate thread to run the fit method
        from threading import Thread
        fit_thread = Thread(target=random_search.fit, args=(X_train, y_train))
        fit_thread.start()
        
        # Check if the time limit has been exceeded
        while fit_thread.is_alive():
            if datetime.now() - start_time > time_limit:
                print("\nTime limit exceeded for {}. Moving to next model.".format(setup_name))
                raise TimeoutError
            time.sleep(10)  # Check every 10 seconds
        
        fit_thread.join()
        
    except TimeoutError:
        continue
    except Exception as e:
        print("An error occurred with {}: {}".format(setup_name, str(e)))
        continue
    
    end_time = datetime.now()
    time_taken = (end_time - start_time).total_seconds()
    
    _results = {
        'name': setup_name,
        'roc_auc': random_search.cv_results_['mean_test_roc_auc'][random_search.best_index_],
        'accuracy': random_search.cv_results_['mean_test_accuracy'][random_search.best_index_],
        'precision': random_search.cv_results_['mean_test_precision'][random_search.best_index_],
        'recall': random_search.cv_results_['mean_test_recall'][random_search.best_index_],
        'f1': random_search.cv_results_['mean_test_f1'][random_search.best_index_],
        'time_taken': time_taken
    }
    
    # Pickle best parameters with timestamp
    param_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    param_file_name = '{}/{}_{}_cv_results_{}.pkl'.format(CROSS_VALIDATION_DIR, run_id, setup_name, param_timestamp)
    
    with open(param_file_name, 'wb') as f:
        pickle.dump({'params': random_search.best_params_, 'result': _results}, f)
        
    print(f"Time taken: {time_taken:.2f} seconds")

In [ ]:
random_search_results_df, best_params = compile_cv_results(3)
random_search_results_df

In [ ]:
best_models_df = get_best_models(random_search_results_df)
best_models_df

In [ ]:
best_models_list = best_models_df['name'].values.tolist()
best_models_list

# Model evaluation

- Training the model on the training data to learn the underlying patterns
- Evaluating and selecting the best model based on performance metrics and their relative weights
- Presenting the model's performance, insights, and key metrics in a clear and meaningful way

### Section helper methods

In [ ]:
# Define a function for early stopping
def early_stopping_callback(estimator, X, y, cv, n_iter_no_change=10, tol=1e-4):
    scores = []
    for i, (train, val) in enumerate(cv.split(X, y)):
        estimator.fit(X.iloc[train], y.iloc[train])
        score = estimator.score(X.iloc[val], y.iloc[val])
        scores.append(score)
        if i >= n_iter_no_change and np.mean(scores[-n_iter_no_change:]) < np.max(scores[:-n_iter_no_change]) + tol:
            break
            
    return estimator

In [ ]:
def load_model_fit_metrics(_model_names: list, _fit_id: int) -> DataFrame:
    metrics_list: list = list()
    for model_name in _model_names:
        pickle_file_name = '{}/{}_{}_metrics.pkl'.format(MODEL_FITTING_RESULTS_DIR, _fit_id, model_name)
        with open(pickle_file_name, 'rb') as f:
            metrics = pickle.load(f)
            metrics_list.append(metrics)
        f.close()
    return DataFrame(metrics_list)

### Model fitting

In [ ]:
# Iterate through all selected models
fit_id = 0
for model_details in modelling_setup:
    model_name = model_details.get('name')

    if model_name not in best_models_list:
        continue

    print('\n{}-------------'.format(model_name))
    clf_model = model_details.get('classifier')
    model_hyper_parameters = best_params.get(model_name)

    # Set the best parameters for the model
    tuned_model = clf_model.set_params(**model_hyper_parameters)

    # Record the start time
    start_time = time.time()

    # Fit training data to the tuned model with early stopping for tree-based models
    if model_name in ['xgb', 'lgbm']:
        # For tree-based models, try to use early stopping if available
        try:
            tuned_model.fit(X_train, y_train, 
                            eval_set=[(X_test, y_test)],
                            early_stopping_rounds=10,
                            verbose=False)
        except TypeError:
            # If early stopping is not available, fall back to regular fit
            tuned_model.fit(X_train, y_train)
            
    else:
        # For other models, just fit without early stopping
        tuned_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = tuned_model.predict(X_test)
    y_pred_proba = tuned_model.predict_proba(X_test)[:, 1] if hasattr(tuned_model, 'predict_proba') else y_pred

    # Calculate the metrics
    _accuracy = accuracy_score(y_test, y_pred)
    _precision = precision_score(y_test, y_pred)
    _recall = recall_score(y_test, y_pred)
    _f1 = f1_score(y_test, y_pred)
    _auc = roc_auc_score(y_test, y_pred_proba)

    # Calculate the time taken
    _time_taken = time.time() - start_time

    # Store the metrics in a dictionary
    metrics = {
        'model name': model_name,
        'accuracy': _accuracy,
        'precision': _precision,
        'recall': _recall,
        'f1': _f1,
        'roc_auc': _auc,
        'time taken (seconds)': _time_taken
    }

    # Pickle the tuned model
    with open('{}/{}_{}_tuned_model.pkl'.format(MODEL_FITTING_RESULTS_DIR, fit_id, model_name), 'wb') as f:
        pickle.dump(tuned_model, f)

    # Pickle the metrics
    with open('{}/{}_{}_metrics.pkl'.format(MODEL_FITTING_RESULTS_DIR, fit_id, model_name), 'wb') as f:
        pickle.dump(metrics, f)

    print(f"Model {model_name} trained, evaluated, and saved.")

### Model selection
Thought process
- Metric weights
- Met

In [ ]:
# Step 1: Load pickled results and display as dataframe
model_fit_metrics_df = load_model_fit_metrics(best_models_list, 0)
model_fit_metrics_df

In [ ]:
# Step 2: Rank models and get the best 2
best_fitted_models = get_best_models(model_fit_metrics_df, 2)
best_fitted_models

In [ ]:
top_2_models = best_fitted_models['model name'].values.tolist()
top_2_models

In [ ]:
# Step 3: Test for overfitting and underfitting
def evaluate_model(model, _X_train, _X_test, _y_train, _y_test):
    train_pred = model.predict(_X_train)
    test_pred = model.predict(_X_test)
    train_auc = roc_auc_score(_y_train, model.predict_proba(_X_train)[:, 1])
    test_auc = roc_auc_score(_y_test, model.predict_proba(_X_test)[:, 1])
    return {
        'train_accuracy': accuracy_score(y_train, train_pred),
        'test_accuracy': accuracy_score(y_test, test_pred),
        'train_auc': train_auc,
        'test_auc': test_auc
    }

In [ ]:
fit_id = 0
evaluation_results = {}
for model_name in top_2_models:
    pickled_file_name = '{}/{}_{}_tuned_model.pkl'.format(MODEL_FITTING_RESULTS_DIR, fit_id, model_name)
    with open(pickled_file_name, 'rb') as f:
        model = pickle.load(f)
    f.close()
    evaluation_results[model_name] = evaluate_model(model, X_train, X_test, y_train, y_test)

print("\nEvaluation Results:")
DataFrame(evaluation_results).T

In [ ]:
# Additional: Calculate and display Kolmogorov-Smirnov statistic
def ks_statistic(y_true, y_pred_proba):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    return max(tpr - fpr)

ks_stat = ks_statistic(y_test, y_pred_proba)
print(f"\nKolmogorov-Smirnov statistic: {ks_stat:.4f}")

In [ ]:
# Select the best model based on smallest gap between train and test AUC
best_model_name = min(evaluation_results, key=lambda x: abs(evaluation_results[x]['train_auc'] - evaluation_results[x]['test_auc']))
print("\nBest model: {}".format(best_model_name))

In [ ]:
with open('{}/{}_{}_tuned_model.pkl'.format(MODEL_FITTING_RESULTS_DIR, fit_id, best_model_name), 'rb') as f:
    loan_default_model = pickle.load(f)
f.close()
loan_default_model

### Model results display

In [ ]:
# Make predictions on the validation set
y_pred_val = loan_default_model.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred_val)

_viz_name = 'eda/confusion matrix.png'
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig(_viz_name)
# Display the image
# Image(filename=_viz_name)

In [ ]:
# Make predictions on the validation set
y_pred_val_proba = loan_default_model.predict_proba(X_test)[:, 1]  # Get the probability of default

# Calculate the ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_val_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
_viz_name = 'eda/roc.png'
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig(_viz_name)
# Display the image
# Image(filename=_viz_name)

In [ ]:
# Plot the feature importance
feature_importance = loan_default_model.feature_importances_
feature_names = X_test.columns

_viz_name = 'eda/feature importance.png'
# 
plt.figure(figsize=(10, 20))
plt.barh(feature_names, feature_importance)
# plt.xticks(rotation=60)
plt.ylabel('Feature')
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.savefig(_viz_name)
# Display the image
# Image(filename=_viz_name)

# Save model

- Training and storing the model and associated artifacts for future use, deployment, or further refinement.

## Section helper methods

In [ ]:
def save_trained_model(_model, _model_name: str='loan_default_prediction'):
    # Check for any models in model deployment directory
    model_list = listdir(MODEL_DEPLOYMENT_DIR)

    # When model list exists
    check_name_count: int = 0
    if model_list:
        check_name_list = [model_name_ for model_name_ in model_list if _model_name in model_name_]
        check_name_count = len(check_name_list)

    # Indicate the nth of the coming report
    check_name_count += 1
    model_file_name = '{}/{}_{}.pkl'.format(MODEL_DEPLOYMENT_DIR, _model_name, check_name_count)
    
    # Save the model to a file
    joblib.dump(_model, model_file_name)

    # When report has been generated
    if exists(model_file_name):
        print('Model: {} successfully saved!'.format(model_file_name))

    else:
        print('No generated model!')

## Model saving process

In [ ]:
save_trained_model(loan_default_model)